In [1]:
import pandas as pd
import numpy as np

# location processing
# from geotext import GeoText
from geopy import geocoders  
# import geocoder
# import requests

import multiprocessing as mp
from multiprocessing import Pool, Array, Lock
from time import time
from joblib import delayed, Parallel

In [2]:
all_data = pd.read_csv('../complete_data/complete_data_sentiment.csv')
all_data.head(1)

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,Total_Test_Results,Case_Fatality_Ratio,Testing_Rate,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0


In [140]:
# returns coordinates given location
def get_coords2(i,loc,gn):
    if i % 50 == 0:
        print('location',i,':',loc,'percent:',round(i/12940*100,2))
    try:
        location = gn.geocode(loc,exactly_one=False)
        coords = location[0][1]
    except Exception as e:
        print('invalid location',i,':',loc)
        coords = (None,None) # don't need to return coords
        # return
    return coords

In [51]:
# sort by most common locations -> get unique locations for geocoder
sorted_unique_locs = list(all_data.location.value_counts().index)
# unique_locs = all_data.location.unique()
coord_dict_not_none = dict(filter(lambda elem: elem[1] != (None,None),coord_dict.items()))
already_retrieved = list(coord_dict_not_none.keys())
already_retrieved

sorted_unique_locs_left = [l for l in sorted_unique_locs if l not in already_retrieved]
print(len(coord_dict_not_none), len(sorted_unique_locs),len(sorted_unique_locs_left))
# num_unique_loc = len(sorted_unique_locs)
# unique_locs[30:50]

912 13852 12940


In [53]:
# test rate limit
gn = geocoders.GeoNames(username='tylerwu')
get_coords2(1,'CA, US', gn)
# sorted_unique_locs_left

(8.62, -170.11)

In [42]:

# location = gn.geocode('CA, US',exactly_one=False)[0][1]
# location

CA, US the hourly limit of 1000 credits for tylerwu has been exceeded. Please throttle your requests or use the commercial service.


(45.14901, -122.6512)

In [21]:
# coord_dict = {}

In [185]:
# pool = mp.Pool()
# NUM_CPUS=8

# index = range(0,len(unique_locs))
# step = 1900
# index = range(step,step+1000)
# index = range(0,len(remaining_coords))
# coord_dict = [pool.apply(get_coords2, args=(i, unique_locs[i], gn)) for i in index[:32]] # mp package
# coord_dict = Parallel(n_jobs=NUM_CPUS)(delayed(get_coords2)(i,unique_locs[i],gn) for i in index[:10]) #joblib package
# some_coords_dict = {sorted_unique_locs_left[i]:get_coords2(i,sorted_unique_locs_left[i], gn) for i in index} # old-fashioned way
some_coords_dict = {some_coords:get_coords2(0,some_coords, gn) for some_coords in sorted_unique_locs[12900:]} # old-fashioned way
# some_coords_dict = {remaining_coords[i]:get_coords2(i,remaining_coords[i], gn) for i in index} # old-fashioned way
# # pool.close()
some_coords_dict

location 0 : MontrÃ©al, CA percent: 0.0
invalid location 0 : MontrÃ©al, CA
location 0 : Eagle Springs (Atascocita), TX percent: 0.0
invalid location 0 : Eagle Springs (Atascocita), TX
location 0 : Woodlyn, PA percent: 0.0
location 0 : New Oxford, PA percent: 0.0
location 0 : Donaldsonville, LA percent: 0.0
location 0 : Cambridge, Ohio percent: 0.0
location 0 : Dawson, IL percent: 0.0
location 0 : Germantown, Wi percent: 0.0
location 0 : Winter Spring, FL percent: 0.0
location 0 : Pleasure Point Santa Cruz, CA percent: 0.0
location 0 : Northeastern, PA percent: 0.0
location 0 : Florida man | Washington, DC percent: 0.0
invalid location 0 : Florida man | Washington, DC
location 0 : Duvall, WA percent: 0.0
location 0 : Chelsea, MI percent: 0.0
location 0 : Fremont, OH percent: 0.0
location 0 : Niantic, CT percent: 0.0
location 0 : Stanton, Delaware, USA percent: 0.0
location 0 : metro Detroit, Michigan percent: 0.0
location 0 : Stoneville, NC percent: 0.0
location 0 : TP, IL percent: 0.0


{'MontrÃ©al, CA': (None, None),
 'Eagle Springs (Atascocita), TX': (None, None),
 'Woodlyn, PA': (39.87233, -75.33713),
 'New Oxford, PA': (44.25979, -70.50062),
 'Donaldsonville, LA': (30.10114, -90.99412),
 'Cambridge, Ohio': (40.05205, -81.49426),
 'Dawson, IL': (39.85283, -89.46343),
 'Germantown, Wi': (43.22862, -88.11037),
 'Winter Spring, FL': (28.03823, -81.73506),
 'Pleasure Point Santa Cruz, CA': (36.95412, -121.97579),
 'Northeastern, PA': (41.30225, -76.02801),
 'Florida man | Washington, DC': (None, None),
 'Duvall, WA': (47.74232, -121.98568),
 'Chelsea, MI': (42.31807, -84.02181),
 'Fremont, OH': (41.35742, -83.14392),
 'Niantic, CT': (41.32538, -72.19313),
 'Stanton, Delaware, USA': (39.71567, -75.64076),
 'metro Detroit, Michigan': (42.33143, -83.04575),
 'Stoneville, NC': (36.46653, -79.90698),
 'TP, IL': (38.17417, 12.7363),
 'Randolph Air Force Base, TX': (29.52967, -98.27918),
 'Isle of Misfit Black Girls, PA': (None, None),
 'Uniondale, PA': (40.70038, -73.59291),

In [ ]:
remaining_coords = list(dict(filter(lambda elem: elem[1] == (None,None),some_coords_dict.items())).keys())
len(remaining_coords)
remaining_coords

In [186]:
# print(len(some_coords_dict))
print(len(coord_dict))

11374


In [187]:
coord_dict.update(some_coords_dict)
print(len(coord_dict))

11582


In [188]:
coord_dict =  dict(filter(lambda elem: elem[1] != (None,None) and elem[1] != None,coord_dict.items()))
print(len(coord_dict))

11409


In [88]:
res = None in coord_dict.values()
res

False

In [189]:
# sub_data = all_data.head()
coord_dict_lat = {coord:coord_dict[coord][0] for coord in coord_dict}
coord_dict_long = {coord:coord_dict[coord][1] for coord in coord_dict}
# coord_dict_lat
all_data['coordinates'] = all_data['location'].map(coord_dict)
all_data['latitude'] = all_data['location'].map(coord_dict_lat)
all_data['longitude'] = all_data['location'].map(coord_dict_long)
all_data.head()

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2,latitude,longitude,coordinates
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,0.000,0.000,0.0000,0,0,34.05529,-117.75228,"(34.05529, -117.75228)"
1,1.250000e+18,"Santa Barbara, CA",4/21/2020,34308,5245,XO face masks are back to raise money and Iâ...,2,Santa Barbara,California,False,...,300100.0,3.607815,0.067,0.080,0.1280,1,1,34.42083,-119.69819,"(34.42083, -119.69819)"
2,1.250000e+18,"California, USA",4/21/2020,0,0,@ImKingGinger Well aren't you special? My rela...,3,Not City,California,True,...,300100.0,3.607815,0.258,0.052,-0.8104,-1,-1,34.68743,-116.78467,"(34.68743, -116.78467)"
3,1.250000e+18,"Arizona, USA",4/19/2020,4,0,@seanhannity If I was to stay home and eat ice...,4,Not City,Arizona,True,...,65652.0,3.729982,0.162,0.110,-0.4497,-1,-1,34.50030,-111.50098,"(34.5003, -111.50098)"
4,1.250000e+18,"Medford, MA",4/6/2020,0,0,"@MisagaLion @tedpanos Yup, CT. Governor spoke ...",5,Medford,Massachusetts,False,...,NaN,NaN,0.224,0.108,-0.6988,-1,-1,42.41843,-71.10616,"(42.41843, -71.10616)"


In [190]:
all_data.coordinates.value_counts()
# sum(all_data.coordinates.isna())

(40.71427, -74.00597)     5028
(34.68743, -116.78467)    4426
(34.05223, -118.24368)    3879
(38.89511, -77.03637)     3722
(41.85003, -87.65005)     3274
                          ... 
(40.55782, -90.67346)        1
(40.35399, -75.3049)         1
(40.17844, -75.12851)        1
(31.1024, -85.69688)         1
(37.1795, -85.90692)         1
Name: coordinates, Length: 7386, dtype: int64

In [191]:
all_data.to_csv('../complete_data/complete_data.csv',index=False)

In [192]:
all_data = pd.read_csv('../complete_data/complete_data.csv')
all_data.head()

,tweet_id,location,date,likes,retweets,text,index,city,state,is_state,...,People_Tested,Mortality_Rate,polarity_neg,polarity_pos,polarity_compound,polarity_classif,polarity_classif2,latitude,longitude,coordinates
0,1.250000e+18,"Pomona, CA",4/4/2020,89921,34665,A day without Hispanics .... this country coul...,1,Pomona,California,False,...,NaN,NaN,0.000,0.000,0.0000,0,0,34.05529,-117.75228,"(34.05529, -117.75228)"
1,1.250000e+18,"Santa Barbara, CA",4/21/2020,34308,5245,XO face masks are back to raise money and Iâ...,2,Santa Barbara,California,False,...,300100.0,3.607815,0.067,0.080,0.1280,1,1,34.42083,-119.69819,"(34.42083, -119.69819)"
2,1.250000e+18,"California, USA",4/21/2020,0,0,@ImKingGinger Well aren't you special? My rela...,3,Not City,California,True,...,300100.0,3.607815,0.258,0.052,-0.8104,-1,-1,34.68743,-116.78467,"(34.68743, -116.78467)"
3,1.250000e+18,"Arizona, USA",4/19/2020,4,0,@seanhannity If I was to stay home and eat ice...,4,Not City,Arizona,True,...,65652.0,3.729982,0.162,0.110,-0.4497,-1,-1,34.50030,-111.50098,"(34.5003, -111.50098)"
4,1.250000e+18,"Medford, MA",4/6/2020,0,0,"@MisagaLion @tedpanos Yup, CT. Governor spoke ...",5,Medford,Massachusetts,False,...,NaN,NaN,0.224,0.108,-0.6988,-1,-1,42.41843,-71.10616,"(42.41843, -71.10616)"


In [193]:
all_data.columns

Index(['tweet_id', 'location', 'date', 'likes', 'retweets', 'text', 'index',
       'city', 'state', 'is_state', 'state_abbr', 'city_and_state',
       'State_Tribe_Territory', 'Face_Masks_Required_in_Public', 'Polarity',
       'Subjectivity', 'CA', 'LA', 'tokens', 'mentions_mask', 'Unnamed: 0',
       'Province_State', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'Incident_Rate', 'Total_Test_Results', 'Case_Fatality_Ratio',
       'Testing_Rate', 'People_Tested', 'Mortality_Rate', 'polarity_neg',
       'polarity_pos', 'polarity_compound', 'polarity_classif',
       'polarity_classif2', 'latitude', 'longitude', 'coordinates'],
      dtype='object')

In [194]:
all_data = all_data.drop(['tweet_id','index','Unnamed: 0'],axis=1)
all_data.to_csv('../complete_data/complete_data.csv',index=False)

KeyError: "['tweet_id' 'index' 'Unnamed: 0'] not found in axis"